# CORD-19-analyse-collected-data

In general, this jupyter notebook is designated to analyse additional colllected scopus data. Thusly, an API-call is described: 
https://api.elsevier.com/content/search/scopus?query=DOI(10.1109/MCOM.2016.7509373)&apiKey=6d485ef1fe1408712f37e8a783a285a4

First, relevant packages must be imported to the Notebook.

In [46]:
import numpy as np
import pandas as pd
import csv
import ast
import collections
import matplotlib.pyplot as plt
import Levenshtein as lev
from fuzzywuzzy import fuzz 
import datetime
import matplotlib.pyplot as plt
import re
from urllib.parse import urlparse
from collections import Counter

from elsapy.elsclient import ElsClient
from elsapy.elsdoc import FullDoc, AbsDoc
from elsapy.elssearch import ElsSearch

import time # for sleep
from pybtex.database import parse_file, BibliographyData, Entry
import json
from elsapy.elsclient import ElsClient
from elsapy.elsdoc import AbsDoc
from elsapy.elssearch import ElsSearch

Thusly, the fetched coredata and affiliation are read from the pkl-files and combined to a DataFrame for further processing. 

In [47]:
read_affiliation = pd.read_pickle('extra_info_affiliation_CS5099.pkl')
read_coredata = pd.read_pickle('extra_info_coredata_CS5099.pkl')
df_current_extra_info = pd.DataFrame()
df_current_extra_info['affiliation'] = read_affiliation
df_current_extra_info['coredata'] = read_coredata
df_current_extra_info

,affiliation,coredata
0,"[{'affiliation-city': None, 'affilname': 'Pata...","{'srctype': 'j', 'prism:issueIdentifier': '230..."
1,"[{'affiliation-city': 'New Delhi', 'affilname'...","{'srctype': 'j', 'eid': '2-s2.0-85083171050', ..."
2,None,None
3,"[{'affiliation-city': 'London', 'affilname': '...","{'srctype': 'j', 'eid': '2-s2.0-79953057246', ..."
4,"[{'affiliation-city': 'Baoding', 'affilname': ...","{'srctype': 'j', 'prism:issueIdentifier': '7',..."
...,...,...
74297,"[{'affiliation-city': 'Wageningen', 'affilname...","{'srctype': 'j', 'prism:issueIdentifier': '4',..."
74298,"[{'affiliation-city': 'London', 'affilname': '...","{'srctype': 'j', 'prism:issueIdentifier': '12 ..."
74299,"{'affiliation-city': 'Adelaide', 'affilname': ...","{'srctype': 'j', 'eid': '2-s2.0-85091772561', ..."
74300,"[{'affiliation-city': 'San Diego', 'affilname'...","{'srctype': 'j', 'prism:issueIdentifier': '6',..."


In [65]:
df_current_extra_info.isnull().sum()

affiliation    14615
coredata       13897
dtype: int64

Thusly, all rows which contain "None" values are dropped and the DataFrame is reindexed. 

In [19]:
df_combined = df_current_extra_info.dropna()
df_combined = df_combined.reset_index(drop=True)
df_combined

,affiliation,coredata
0,"[{'affiliation-city': None, 'affilname': 'Pata...","{'srctype': 'j', 'prism:issueIdentifier': '230..."
1,"[{'affiliation-city': 'New Delhi', 'affilname'...","{'srctype': 'j', 'eid': '2-s2.0-85083171050', ..."
2,"[{'affiliation-city': 'London', 'affilname': '...","{'srctype': 'j', 'eid': '2-s2.0-79953057246', ..."
3,"[{'affiliation-city': 'Baoding', 'affilname': ...","{'srctype': 'j', 'prism:issueIdentifier': '7',..."
4,"{'affiliation-city': 'Brighton', 'affilname': ...","{'srctype': 'j', 'eid': '2-s2.0-85091848821', ..."
...,...,...
59682,"[{'affiliation-city': 'Wageningen', 'affilname...","{'srctype': 'j', 'prism:issueIdentifier': '4',..."
59683,"[{'affiliation-city': 'London', 'affilname': '...","{'srctype': 'j', 'prism:issueIdentifier': '12 ..."
59684,"{'affiliation-city': 'Adelaide', 'affilname': ...","{'srctype': 'j', 'eid': '2-s2.0-85091772561', ..."
59685,"[{'affiliation-city': 'San Diego', 'affilname'...","{'srctype': 'j', 'prism:issueIdentifier': '6',..."


The following functions support the creation of DataFrames based on the columns affiliation and coredata. 

In [20]:
def get_one_entry(dic):
    df_affiliation_holder = pd.DataFrame(dic.items()).T
    df_affiliation_holder.columns = df_affiliation_holder.iloc[0]
    df_affiliation_holder = df_affiliation_holder.drop(df_affiliation_holder.index[0])
    return df_affiliation_holder

In [21]:
def get_various_entries(dic):
    df_affiliation_holder = pd.DataFrame.from_dict(dic, orient='columns')
    return df_affiliation_holder

The next cell creates the DataFrame which focus on affiliation data. 

In [22]:
%%time
df_affiliation = pd.DataFrame()
df_affiliation_holder = pd.DataFrame()

for i in df_combined['affiliation']:
    string_holder = str(i)
    if string_holder[0] == "[":
        df_affiliation_holder = get_various_entries(i)
    else: 
        df_affiliation_holder = get_one_entry(i)
    df_affiliation = pd.concat([df_affiliation_holder, df_affiliation],ignore_index=True)
df_affiliation

Wall time: 21min 42s


,affiliation-city,affilname,affiliation-country
0,Taipei,Ming Chi University of Technology,Taiwan
1,Taipei,National Taiwan University of Science and Tech...,Taiwan
2,Chung-Li,Yuan Ze University,Taiwan
3,San Diego,"University of California, San Diego",United States
4,Pittsburgh,University of Pittsburgh,United States
...,...,...,...
198889,New Delhi,Jamia Millia Islamia,India
198890,New Delhi,Indraprastha Apollo Hospitals,India
198891,None,Patan Academy of Health Sciences-School of Med...,Nepal
198892,Valhalla,New York Medical College,United States


In [19]:
# %%time
# df_coredata = pd.DataFrame()
# df_coredata_holder = pd.DataFrame()

# for i in df_combined['coredata']:
#     string_holder = str(i)
#     if string_holder[0] == "[":
#         df_coredata_holder = get_various_entries(i)
#     else: 
#         df_coredata_holder = get_one_entry(i)
#     df_coredata = pd.concat([df_coredata_holder, df_coredata],ignore_index=True)
# df_coredata

Wall time: 4h 14min 18s


,srctype,prism:issueIdentifier,eid,prism:coverDate,prism:aggregationType,prism:url,subtypeDescription,dc:creator,link,prism:publicationName,...,prism:issn,article-number,dc:identifier,dc:publisher,pubmed-id,prism:pageRange,prism:endingPage,prism:startingPage,pii,prism:isbn
0,j,5,2-s2.0-85057840602,2018-04-26,Journal,https://api.elsevier.com/content/abstract/scop...,Article,"{'author': [{'ce:given-name': 'Chien-Chih', 'p...","[{'@_fa': 'true', '@rel': 'self', '@href': 'ht...",Entropy,...,10994300,320,SCOPUS_ID:85057840602,MDPI AGPostfachBaselCH-4005indexing@mdpi.com,NaN,NaN,NaN,NaN,NaN,NaN
1,j,6,2-s2.0-85097986550,2020-12-01,Journal,https://api.elsevier.com/content/abstract/scop...,Review,"{'author': [{'ce:given-name': 'Zaid', 'preferr...","[{'@_fa': 'true', '@rel': 'self', '@href': 'ht...",mSystems,...,23795077,00999,SCOPUS_ID:85097986550,American Society for Microbiology,NaN,NaN,NaN,NaN,NaN,NaN
2,j,6,2-s2.0-85091772561,2021-06-01,Journal,https://api.elsevier.com/content/abstract/scop...,Article,"{'author': [{'ce:given-name': 'Richard', 'pref...","[{'@_fa': 'true', '@rel': 'self', '@href': 'ht...",Addiction,...,13600443 09652140,NaN,SCOPUS_ID:85091772561,Blackwell Publishing Ltd,32945597,1600-1605,1605,1600,NaN,NaN
3,j,12 December,2-s2.0-85097843658,2020-12-01,Journal,https://api.elsevier.com/content/abstract/scop...,Article,"{'author': [{'ce:given-name': 'Martina', 'pref...","[{'@_fa': 'true', '@rel': 'self', '@href': 'ht...",PLoS ONE,...,19326203,e0242914,SCOPUS_ID:85097843658,Public Library of Science,33296395,NaN,NaN,NaN,NaN,NaN
4,j,4,2-s2.0-85091314414,2020-12-01,Journal,https://api.elsevier.com/content/abstract/scop...,Article,"{'author': [{'ce:given-name': 'Linda JL', 'pre...","[{'@_fa': 'true', '@rel': 'self', '@href': 'ht...",Outlook on Agriculture,...,00307270,NaN,SCOPUS_ID:85091314414,SAGE Publications Inc. claims@sagepub.com,NaN,321-329,329,321,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
59682,j,1,2-s2.0-85091848821,2021-02-01,Journal,https://api.elsevier.com/content/abstract/scop...,Note,"{'author': [{'ce:given-name': 'Manahal', 'pref...","[{'@_fa': 'true', '@rel': 'self', '@href': 'ht...",Clinical Teacher,...,1743498X 17434971,NaN,SCOPUS_ID:85091848821,Blackwell Publishing Ltd,33006439,90-91,91,90,NaN,NaN
59683,j,7,2-s2.0-85049437388,2018-01-01,Journal,https://api.elsevier.com/content/abstract/scop...,Article,"{'author': [{'ce:given-name': 'Fei', 'preferre...","[{'@_fa': 'true', '@rel': 'self', '@href': 'ht...",Molecules,...,14203049,1599,SCOPUS_ID:85049437388,MDPI AGPostfachBaselCH-4005,30004408,NaN,NaN,NaN,NaN,NaN
59684,j,4,2-s2.0-79953057246,2011-04-01,Journal,https://api.elsevier.com/content/abstract/scop...,Review,"{'author': [{'ce:given-name': 'Val', 'preferre...","[{'@_fa': 'true', '@rel': 'self', '@href': 'ht...",The Lancet Infectious Diseases,...,14733099,NaN,SCOPUS_ID:79953057246,NaN,21453872,312-321,321,312,S1473309910702243,NaN
59685,j,4,2-s2.0-85083171050,2020-07-01,Journal,https://api.elsevier.com/content/abstract/scop...,Article,"{'author': [{'ce:given-name': 'Raju', 'preferr...","[{'@_fa': 'true', '@rel': 'self', '@href': 'ht...",Diabetes and Metabolic Syndrome: Clinical Rese...,...,18780334 18714021,NaN,SCOPUS_ID:85083171050,Elsevier Ltd,32305024,337-339,339,337,S1871402120300771,NaN


Leading countries

In [58]:
ser_country = df_affiliation['affiliation-country']
ser_country

0                Taiwan
1                Taiwan
2                Taiwan
3         United States
4         United States
              ...      
198889            India
198890            India
198891            Nepal
198892    United States
198893            India
Name: affiliation-country, Length: 198894, dtype: object

In [59]:
ser_country.value_counts()

United States               43536
China                       22952
United Kingdom              14702
Italy                       10504
France                       8548
                            ...  
Curaçao                         1
Dominica                        1
Mauritania                      1
Samoa                           1
Central African Republic        1
Name: affiliation-country, Length: 191, dtype: int64

Leading cities

In [62]:
ser_country = df_affiliation['affiliation-city']
ser_country

0             Taipei
1             Taipei
2           Chung-Li
3          San Diego
4         Pittsburgh
             ...    
198889     New Delhi
198890     New Delhi
198891          None
198892      Valhalla
198893     New Delhi
Name: affiliation-city, Length: 198894, dtype: object

In [63]:
ser_country.value_counts()

Beijing       5423
London        4752
New York      2767
Paris         2560
Wuhan         2528
              ... 
Torres           1
Zefat            1
Machida          1
West Islip       1
Samara           1
Name: affiliation-city, Length: 5453, dtype: int64

Leading Universities

In [60]:
ser_university = df_affiliation['affilname']
ser_university

0                         Ming Chi University of Technology
1         National Taiwan University of Science and Tech...
2                                        Yuan Ze University
3                       University of California, San Diego
4                                  University of Pittsburgh
                                ...                        
198889                                 Jamia Millia Islamia
198890                        Indraprastha Apollo Hospitals
198891    Patan Academy of Health Sciences-School of Med...
198892                             New York Medical College
198893    All India Institute of Medical Sciences, New D...
Name: affilname, Length: 198894, dtype: object

In [61]:
ser_university.value_counts()

Harvard Medical School                      743
University of Toronto                       575
Tongji Medical College                      572
University College London                   510
University of Washington                    490
                                           ... 
University for Development Studies Ghana      1
Gastrointestinal Surgery Center               1
Emergencias Sanitarias                        1
Summa Health System                           1
Foundation for Global Community Health        1
Name: affilname, Length: 28487, dtype: int64